# Import Library #
- pip install numpy
- pip install matplotlib
- pip install pandas

In [1]:
import numpy as np
import pandas as pd
import pymysql

### Read Data ###

In [13]:
df = pd.read_excel('../../storage/app/public/cme/Cme.xlsx')
df.head()

,id,divisi,abbr_d,wilayah,abbr_w,sto,abbr_s,ruangan,kategori,tipe_perangkat,...,serial_number,kap_tps,kap_tpk,satuan,kondisi,status,periode,qrcode,keterangan,reg_date
0,2615,REGIONAL 5,TR5,MALANG,ML,MALANG KOTA,MLG,R. MITRA,AC SPLIT,AC SPLIT,...,E055728,24000.0,24000.0,BTUH,NORMAL,OPERASI,2008,R503473,SPLIT,2021-06-21 04:33:32
1,2616,REGIONAL 5,TR5,MALANG,ML,MALANG KOTA,MLG,R. BATTERY NEW NGN,AC SPLIT,AC SPLIT,...,37366 7329 INDOOR,18000.0,18000.0,BTUH,NORMAL,OPERASI,2017,R502993,SPLIT,2021-06-21 04:33:32
2,2617,REGIONAL 5,TR5,MALANG,ML,MALANG KOTA,MLG,R. BATTERY NEW NGN,AC SPLIT,AC SPLIT,...,373 663 8247,18000.0,18000.0,BTUH,NORMAL,OPERASI,2017,R503203,SPLIT,2021-06-21 04:33:32
3,2618,REGIONAL 5,TR5,MALANG,ML,MALANG KOTA,MLG,R. DWDM,AC SPLIT,AC SPLIT,...,NaN,24000.0,24000.0,BTUH,NORMAL,OPERASI,2005,R503033,SPLIT,2021-06-21 04:33:32
4,2619,REGIONAL 5,TR5,MALANG,ML,MALANG KOTA,MLG,R.DDF,AC SPLIT,AC SPLIT,...,E0019985,24000.0,24000.0,BTUH,NORMAL,OPERASI,2010,R502733,SPLIT,2021-06-21 04:33:32


In [14]:
stoDevice = df[['sto','kategori']]

stoDevice = df.groupby(['sto', 'kategori']).size().reset_index(name='total per unit')

print(stoDevice)


             sto             kategori  total per unit
0    AMPELGADING             AC SPLIT               6
1    AMPELGADING              BATTERE               3
2    AMPELGADING  DIESEL ENGINE / AMF               4
3    AMPELGADING            GROUNDING               3
4    AMPELGADING       INSTALASI DAYA              10
..           ...                  ...             ...
186        TUREN            GROUNDING               1
187        TUREN       INSTALASI DAYA               3
188        TUREN             INVERTER               1
189        TUREN                OSASE               1
190        TUREN            RECTIFIER               2

[191 rows x 3 columns]


In [15]:
current_year= pd.Timestamp('now').year



df['age'] = current_year - df['periode']

bins = [0, 5, 10, df['age'].max()]
labels = ['underfive', 'morethanfive', 'morethanten']

# Create a new column in the DataFrame based on the conditions
df['age_range'] = pd.cut(df['age'], bins=bins, labels=labels, right=True)

# Group by 'sto' and 'age_range' and count occurrences
pivot_benar = df.groupby(['sto', 'kategori', 'age_range']).size().unstack(fill_value=0).reset_index()

pivot_benar.head(15)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_5148\3310343470.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pivot_benar = df.groupby(['sto', 'kategori', 'age_range']).size().unstack(fill_value=0).reset_index()


age_range,sto,kategori,underfive,morethanfive,morethanten
0,AMPELGADING,AC PRESISI,0,0,0
1,AMPELGADING,AC SPLIT,2,0,4
2,AMPELGADING,BATTERE,0,1,2
3,AMPELGADING,CAPASITOR BANK,0,0,0
4,AMPELGADING,DIESEL ENGINE / AMF,1,0,3
5,AMPELGADING,DIESEL MOBILE,0,0,0
6,AMPELGADING,GROUNDING,0,0,3
7,AMPELGADING,INSTALASI DAYA,0,0,10
8,AMPELGADING,INVERTER,0,0,1
9,AMPELGADING,OSASE,0,0,1


In [16]:
pivot_benar = pd.merge(stoDevice, pivot_benar, on=['sto', 'kategori'], how='left')

pivot_benar.head(15)

,sto,kategori,total per unit,underfive,morethanfive,morethanten
0,AMPELGADING,AC SPLIT,6,2,0,4
1,AMPELGADING,BATTERE,3,0,1,2
2,AMPELGADING,DIESEL ENGINE / AMF,4,1,0,3
3,AMPELGADING,GROUNDING,3,0,0,3
4,AMPELGADING,INSTALASI DAYA,10,0,0,10
5,AMPELGADING,INVERTER,1,0,0,1
6,AMPELGADING,OSASE,1,0,0,1
7,AMPELGADING,RECTIFIER,2,0,0,2
8,BANTUR,AC SPLIT,4,0,0,4
9,BANTUR,BATTERE,4,2,0,2


yang benar

In [17]:
db_host = 'localhost'
db_user = 'root'
db_password = ''
db_name = 'arnet'

# Establish the database connection
connection = pymysql.connect(
    host=db_host,
    user=db_user,
    password=db_password,
    database=db_name
)

exception_mapping = {
    'sumber manjing wetan': 'sumber manjing'
}

def apply_exceptions(value):
    return exception_mapping.get(value.lower(), value)

def get_sto_from_dropdown(value):
    try:
        cursor = connection.cursor()
        # Normalize whitespace and special characters in the input value
        value = apply_exceptions(value)
        
        normalized_value = value.replace(' ', '').replace('/', '').replace('\t', '')
        
        
        # Define the SQL query with a single placeholder for parameterized input
        query = """
        SELECT id 
        FROM dropdowns 
        WHERE type = 'sto' 
          AND REPLACE(REPLACE(REPLACE(subtype, ' ', ''), '/', ''), '\t', '') LIKE %s
        """
        
        # Execute the query with the normalized value wrapped with '%' for LIKE pattern matching
        cursor.execute(query, ('%' + normalized_value + '%',))
        result = cursor.fetchone()
        cursor.close()
        
        if result:
            return result[0]
        else:
            print(f"No 'sto' ID found for value: {value}")
            return None
    except pymysql.MySQLError as e:
        print(f"Error fetching 'sto' ID: {e}")
        return None

    
def get_category_from_dropdown(value):
    try:
        cursor = connection.cursor()
        # Normalize whitespace and special characters in the input value
        normalized_value = value.replace(' ', '').replace('/', '').replace('\t', '')
        
        # Define the SQL query with a single placeholder for parameterized input
        query = """
        SELECT id 
        FROM dropdowns 
        WHERE type = 'category' 
          AND REPLACE(REPLACE(REPLACE(subtype, ' ', ''), '/', ''), '\t', '') LIKE %s
        """
        
        # Execute the query with the normalized value wrapped with '%' for LIKE pattern matching
        cursor.execute(query, ('%' + normalized_value + '%',))
        result = cursor.fetchone()
        cursor.close()
        
        if result:
            return result[0]
        else:
            return None
    except pymysql.MySQLError as e:
        print(f"Error fetching 'category' ID: {e}")
        return None


try:
    cursor = connection.cursor()

    # Truncate the table to clear existing data
    cursor.execute("TRUNCATE TABLE cmes") 
    
    # Define the SQL INSERT statement
    insert_query = """
    INSERT INTO cmes (sto_id, type_id, underfive, morethanfive, morethanten, count)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    
    # Iterate over the DataFrame rows and insert each row into the table
    for index, row in pivot_benar.iterrows():
        sto_id = get_sto_from_dropdown(row['sto'])
        type_id = get_category_from_dropdown(row['kategori'])
        if sto_id is not None:
            cursor.execute(insert_query, (            
                sto_id,
                type_id,
                row['underfive'],
                row['morethanfive'],
                row['morethanten'],
                row['total per unit']
            ))
        else:
            print(f"Warning: No 'sto' ID found for value: {row['sto']}")
    
    # Commit the transaction
    connection.commit()

except pymysql.MySQLError as e:
    print(f"Error during database operations: {e}")

finally:
    # Close the cursor and the connection
    cursor.close()
    connection.close()

print("Data inserted successfully")


Data inserted successfully


In [18]:
# df = pd.read_excel('../../storage/app/public/cme/Data Potensi CME Malang Juli 2024.xlsx')
# df.head()

# Making Pivot #

In [19]:
# category_counts = df.groupby(['sto', 'kategori']).size().reset_index(name='total')

# # Pivot the table
# pivot_table = category_counts.pivot_table(index='sto', columns='kategori', values='total', fill_value=0)

# pivot_table = pivot_table.astype(int)

# pivot_table['Total'] = pivot_table.sum(axis=1)
# # Reset index to make 'sto' a column again
# pivot_table = pivot_table.reset_index()


# pivot_table.head(15)



## Sorting the year ##

#### Converting the Age ####

In [20]:
# current_year=2024

# df['age'] = current_year - df['periode']


#### Count the Number of Products in Each Age Range ####

In [21]:
# bins = [0, 5, 10, df['age'].max()]
# labels = ['underfive', 'morethanfive', 'morethanten']

# # Create a new column in the DataFrame based on the conditions
# df['age_range'] = pd.cut(df['age'], bins=bins, labels=labels, right=True)

# # Group by 'sto' and 'age_range' and count occurrences
# age_range_counts = df.groupby(['sto', 'age_range']).size().unstack(fill_value=0).reset_index()

# print(age_range_counts)

## Combining Tables ##

In [22]:
# combined_df = pd.merge(pivot_table, age_range_counts, on='sto')

# combined_df.head(40)

### Importing To MySql ###

In [23]:
# db_host = 'localhost'
# db_user = 'root'
# db_password = ''
# db_name = 'arnet'

# connection = pymysql.connect(
#     host=db_host,
#     user=db_user,
#     password=db_password,
#     database=db_name
# )

# # Step 1: Mapping 'sto' and 'category' to their respective ids from the 'dropdown' table
# sto_mapping = {}
# category_mapping = {}

# def get_sto_from_dropdown(value):
#     cursor = connection.cursor()
#     query = "SELECT id FROM dropdowns WHERE type = 'sto' AND subtype LIKE %s"
#     cursor.execute(query, ('%' + value + '%',))
#     result = cursor.fetchone()
#     cursor.close()
#     if result:
#         return result[0]
#     else:
#         return None
    
    
# def get_category_from_dropdown(value):
#     cursor = connection.cursor()
#     query = "SELECT id FROM dropdowns WHERE type = 'category' AND subtype LIKE %s"
#     cursor.execute(query, ('%' + value + '%',))
#     result = cursor.fetchone()
#     cursor.close()
#     if result:
#         return result[0]
#     else:
#         return None
    


# try:
#     cursor = connection.cursor()    

#     # Truncate the table to clear existing data
#     cursor.execute("TRUNCATE TABLE cmes")
    
#     # Define the SQL INSERT statement
#     insert_query = """
#     INSERT INTO cmes (sto_id, underfive, morethanfive, morethanten, total)
#     VALUES (%s, %s, %s, %s, %s, %s)
#     """
    
    

#     # Iterate over the DataFrame rows and insert each row into the table
#     for index, row in combined_df.iterrows():
#         cursor.execute(insert_query, (
#             # get sto id
#             get_sto_from_dropdown(row['sto']),            
#             row['underfive'],
#             row['morethanfive'],
#             row['morethanten'],
#             row['Total']
#         ))
    
#     # Commit the transaction
#     connection.commit()

# finally:
#     # Close the cursor and the connection
#     cursor.close()
#     connection.close()

# print("Data inserted successfully")

# This is for creating seeder only run if neccesery #

In [24]:
# # Get unique values of 'sto', 'ruangan', and 'kategori' columns
# sto = df['sto'].unique()
# sto = sorted(sto)
# rooms = df['ruangan'].unique()
# category = df['kategori'].unique()

# # Prepare the data for seeder
# dropdowns = {}

# # Create a dictionary with 'sto' values
# for index, s in enumerate(sto):
#     dropdowns[str(index)] = { 
#         'type': ['sto'],
#         'subtype': [s]
#     }

# # Create a dictionary with 'room' values starting from len(sto)
# start_index = len(sto)
# for index, room in enumerate(rooms):
#     dropdowns[str(start_index + index)] = {  
#         'type': ['room'],
#         'subtype': [room]
#     }

# # Create a dictionary with 'category' values starting from len(sto) + len(rooms)
# start_index = len(sto) + len(rooms)
# for index, c in enumerate(category):
#     dropdowns[str(start_index + index)] = {  
#         'type': ['category'],
#         'subtype': [c]
#     }

# # Convert the dictionary to PHP-like array syntax
# def dict_to_php_array(d):
#     php_array = "{\n"
#     for key, value in d.items():
#         php_array += f"    '{key}' => [\n"
#         php_array += f"        'type' => {value['type']},\n"
#         php_array += f"        'subtype' => {value['subtype']},\n"
#         php_array += "    ],\n"
#     php_array += "}\n"
#     return php_array

# php_array_str = dict_to_php_array(dropdowns)
# print(php_array_str)
